### Notice the two separate PYTHON environment variables. The Driver Python belongs to the Workspace, The other one refers to the one on the EMR worker nodes

In [1]:
import os
hdfs_endpoint=os.environ['HDFS_ENDPOINT']
%env PYSPARK_PYTHON /usr/bin/python3
%env PYSPARK_DRIVER_PYTHON /opt/conda/bin/python

env: PYSPARK_PYTHON=/usr/bin/python3
env: PYSPARK_DRIVER_PYTHON=/opt/conda/bin/python


In [2]:
!hdfs dfs -rmr /user/dominospark/mypi/

rmr: DEPRECATED: Please use '-rm -r' instead.
Deleted /user/dominospark/mypi


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType
import random

In [4]:
sparkSession = SparkSession.builder.appName("Calculate Pi using EMR Spark") \
.config("spark.dynamicAllocation.enabled", "false") \
.config("fs.default.name", hdfs_endpoint) \
.getOrCreate()
sc=sparkSession.sparkContext

In [5]:
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1


Calculate Pi and create a simple dataframe

In [6]:


columns = StructType([ StructField("name", StringType(), True),
                      StructField("value", DoubleType(), True)
                    ])

count = sc.parallelize(range(0, 1000),1) \
             .filter(inside).count()
data = [("Pi",4.0 * count/1000)]

df = sparkSession.createDataFrame(data=data, schema=columns)

df.show()

+----+-----+
|name|value|
+----+-----+
|  Pi|3.208|
+----+-----+



Now write this dataframe to **secure** HDFS

In [7]:
outpath = '/user/dominospark/mypi/'
df.write.csv(outpath)

Finally read it from **secure** HDFS

In [10]:
df_load = sparkSession.read.csv(outpath, schema=columns)
df_load.show()

Py4JJavaError: An error occurred while calling o199.showString.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.GatewayConnection.run(GatewayConnection.java:238)
java.lang.Thread.run(Thread.java:748)

In [9]:
sc.stop()

In [10]:
!hdfs dfs -ls /user/dominospark/mypi/

Found 3 items
-rw-r--r--   1 dominospark dominospark          0 2021-08-13 20:47 /user/dominospark/mypi/_SUCCESS
-rw-r--r--   1 dominospark dominospark          0 2021-08-13 20:47 /user/dominospark/mypi/part-00000-67798971-3daa-4f4a-8613-b1c26a9f8ef0-c000.csv
-rw-r--r--   1 dominospark dominospark          9 2021-08-13 20:47 /user/dominospark/mypi/part-00003-67798971-3daa-4f4a-8613-b1c26a9f8ef0-c000.csv
